# Understanding Components of Custom Data Loader in Pytorch

In [ ]:
!pip install kaggle


In [ ]:
import kagglehub

path=kagglehub.dataset_download("mirichoi0218/insurance")
print(path)

100%|██████████| 16.0k/16.0k [00:00<00:00, 18.4MB/s]

Extracting files...
/root/.cache/kagglehub/datasets/mirichoi0218/insurance/versions/1


In [ ]:
import os
os.listdir("/root/.cache/kagglehub/datasets/mirichoi0218/insurance/versions/1")

['insurance.csv']

In [20]:
import pandas as pd
df=pd.read_csv(os.path.join(path,"insurance.csv"))

In [21]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [23]:
'''
X_train_tensor has 1000000 rows  -> To run this we need let's think we need to use 10 gb of memory --> If not suffivient we can say out of memory

if u have that amount of memory but still we had the issue to update the weights and bias using backpropgation it needs to iterate all over the rows it will be impossible


we are teaching human :A book of 1000 pages --> student has issue in 10 th page , then student i am not able to understand.

1000 pages
100 epoch
we need 10 pages feedback

1000/10=100 iterations

each epoch has 100 iterations . 100 epochs *100 iterations
'''

"\nX_train_tensor has 1000000 rows  -> To run this we need let's think we need to use 10 gb of memory --> If not suffivient we can say out of memory\n\nif u have that amount of memory but still we had the issue to update the weights and bias using backpropgation it needs to iterate all over the rows it will be impossible\n\n\nwe are teaching human :A book of 1000 pages --> student has issue in 10 th page , then student i am not able to understand.\n\n1000 pages\n100 epoch\nwe need 10 pages feedback\n\n1000/10=100 iterations\n\neach epoch has 100 iterations . 100 epochs *100 iterations\n"

In [24]:
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split
import torch
import numpy as np

In [25]:
#split the dataset

train_df,test_df=train_test_split(df,test_size=0.2,random_state=42)


In [26]:
label_encoders={}
for col in df.select_dtypes(include=['object']):
  le=LabelEncoder()
  train_df[col]=le.fit_transform(train_df[col])
  test_df[col]=le.transform(test_df[col])
  label_encoders[col]=le

In [45]:
# Features and target
X_train = train_df.drop('charges', axis=1).values
y_train = train_df['charges'].values
X_test = test_df.drop('charges', axis=1).values
y_test = test_df['charges'].values

In [44]:
X_train

,age,sex,bmi,children,smoker,region
560,46,0,19.950,2,0,1
1285,47,0,24.320,0,0,0
1142,52,0,24.860,0,0,2
969,39,0,34.320,5,0,2
486,54,0,21.470,3,0,1
...,...,...,...,...,...,...
1095,18,0,31.350,4,0,0
1130,39,0,23.870,5,0,2
1294,58,1,25.175,0,0,0
860,37,0,47.600,2,1,3


In [28]:
#Normalize the features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [40]:
y_train.values

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [60]:
# Convert into Tensor

X_train_tensor=torch.tensor(X_train,dtype=torch.float32)
y_train_tensor=torch.tensor(y_train,dtype=torch.float32).view(-1,1) # y_train the data will be in pandas. y_train.values only we are getting values
X_test_tensor=torch.tensor(X_test,dtype=torch.float32)
y_test_tensor=torch.tensor(y_test,dtype=torch.float32).view(-1,1)

#

In [62]:
# Define NN
import torch.nn as nn

class SimpleNNRegresionModel(nn.Module):
  def __init__(self,input_dim):
    super(SimpleNNRegresionModel,self).__init__()
    self.network= nn.Sequential(
        nn.Linear(input_dim,64),
        nn.ReLU(),
        nn.Linear(64,128),
        nn.ReLU(),
        nn.Linear(128,1)
    )

  def forward(self,X):
    return self.network(X)

In [65]:
import torch.optim as optim
input_dim=X_train_tensor.shape[1]

model=SimpleNNRegresionModel(input_dim)

In [66]:
# Loss and optimization

criterion=nn.MSELoss()
optimizer=optim.Adam(model.parameters(),lr=0.01)

In [67]:
# Training loop
epochs=1000
for epoch in range(epochs):
  model.train()
  optimizer.zero_grad()
  output=model(X_train_tensor)
  loss=criterion(output,y_train_tensor)
  loss.backward()
  optimizer.step()

  if (epoch+1)%1000==0:
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')


Epoch [1000/1000], Loss: 22733206.0000


1. Dataset torch.utils.data.Dataset # we can create the new dataset. we will tell our pytorch that how we want to retrieve the dataset, how we want to process the dataset
2. DataLoader torch.utils.data.DataLoader # It will helps to wrap the dataset into an iterable object that lpads the data in bunches.

If i want 1000 rows in a single iteration, data loader will wrap the dataset and ask data dataset to provide 1000 ros in single ieration.

In [29]:
# creating our custom dataset in pytorch

#init() - Initialized the datset, loads data, applied preprocessing
# len() - return the total numbers of samples in the datsaet
# getitem()  - it defines how to reterive the single data sample when index is provided

In [30]:
import torch
from torch.utils.data import Dataset,DataLoader

In [46]:
class InsuranceDataset(Dataset):
  def __init__(self,X,y):
    self.X=X
    self.y=y

  def __len__(self):
    return len(self.X)

  def __getitem__(self,index):
    features=torch.tensor(self.X[index],dtype=torch.float32)
    target=torch.tensor(self.y[index],dtype=torch.float32)
    return features,target



In [52]:
X_train.shape

(1070, 6)

In [53]:
1070/32

33.4375

__init__ is the constructor of the class.

It runs automatically when you create an object:

Because X is stored as self.X, which belongs to the object, not just to __init__

In [48]:
dataset= InsuranceDataset(X_train,y_train)

In [49]:
dataloader=DataLoader(dataset,batch_size=32,shuffle=True) # shuffle means random shuffle and num_workers
# print(dataloader)

In [57]:
for batch in dataloader:
    print(batch)
    break

    # batch = (inputs, labels)

[tensor([[36.0000,  1.0000, 31.5000,  0.0000,  0.0000,  3.0000],
        [47.0000,  1.0000, 29.8000,  3.0000,  1.0000,  3.0000],
        [55.0000,  1.0000, 37.3000,  0.0000,  0.0000,  3.0000],
        [43.0000,  0.0000, 32.5600,  3.0000,  1.0000,  2.0000],
        [18.0000,  0.0000, 21.6600,  0.0000,  1.0000,  0.0000],
        [34.0000,  1.0000, 35.8150,  0.0000,  0.0000,  1.0000],
        [51.0000,  0.0000, 37.7300,  1.0000,  0.0000,  2.0000],
        [48.0000,  0.0000, 33.1100,  0.0000,  1.0000,  2.0000],
        [33.0000,  1.0000, 22.7050,  0.0000,  0.0000,  1.0000],
        [24.0000,  1.0000, 40.1500,  0.0000,  1.0000,  2.0000],
        [51.0000,  1.0000, 39.7000,  1.0000,  0.0000,  3.0000],
        [18.0000,  0.0000, 36.8500,  0.0000,  1.0000,  2.0000],
        [21.0000,  1.0000, 31.1000,  0.0000,  0.0000,  3.0000],
        [31.0000,  1.0000, 31.0650,  3.0000,  0.0000,  1.0000],
        [28.0000,  0.0000, 37.6200,  1.0000,  0.0000,  2.0000],
        [51.0000,  0.0000, 36.6700,  2.

In [51]:
for batch_idx,(features,targets) in enumerate(dataloader): # it will create 33 batches
  print(f"Batch Index: {batch_idx+1}")
  print(f"Features Shape: {features.shape}")
  print(f"Targets: {targets.shape}")


Batch Index: 1
Features Shape: torch.Size([32, 6])
Targets: torch.Size([32])
Batch Index: 2
Features Shape: torch.Size([32, 6])
Targets: torch.Size([32])
Batch Index: 3
Features Shape: torch.Size([32, 6])
Targets: torch.Size([32])
Batch Index: 4
Features Shape: torch.Size([32, 6])
Targets: torch.Size([32])
Batch Index: 5
Features Shape: torch.Size([32, 6])
Targets: torch.Size([32])
Batch Index: 6
Features Shape: torch.Size([32, 6])
Targets: torch.Size([32])
Batch Index: 7
Features Shape: torch.Size([32, 6])
Targets: torch.Size([32])
Batch Index: 8
Features Shape: torch.Size([32, 6])
Targets: torch.Size([32])
Batch Index: 9
Features Shape: torch.Size([32, 6])
Targets: torch.Size([32])
Batch Index: 10
Features Shape: torch.Size([32, 6])
Targets: torch.Size([32])
Batch Index: 11
Features Shape: torch.Size([32, 6])
Targets: torch.Size([32])
Batch Index: 12
Features Shape: torch.Size([32, 6])
Targets: torch.Size([32])
Batch Index: 13
Features Shape: torch.Size([32, 6])
Targets: torch.Size([

In [58]:
1070/32
# it will created 33 batches with

33.4375

In [71]:
epochs=1000

for epoch in range(epochs):
  model.train()

  for batch_idx,(batch_x,batch_y) in enumerate(dataloader):
    print(f"Current Batch :{batch_idx}")
    optimizer.zero_grad()
    predictions=model(batch_x)
    loss=criterion(predictions,batch_y)
    loss.backward()
    optimizer.step()
    print(f"Batch[{batch_idx+1}/{epochs}], Loss: {loss.item():.4f}")
    print("="*60)

    if (epoch+1)%100==0:
      print("**"*60)
      print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
      print("**"*60)




Streaming output truncated to the last 5000 lines.
Batch[34/1000], Loss: 100314208.0000
Current Batch :0
Batch[1/1000], Loss: 197072144.0000
Current Batch :1
Batch[2/1000], Loss: 111864072.0000
Current Batch :2
Batch[3/1000], Loss: 165766288.0000
Current Batch :3
Batch[4/1000], Loss: 150586800.0000
Current Batch :4
Batch[5/1000], Loss: 195075552.0000
Current Batch :5
Batch[6/1000], Loss: 132139064.0000
Current Batch :6
Batch[7/1000], Loss: 92802856.0000
Current Batch :7
Batch[8/1000], Loss: 227632464.0000
Current Batch :8
Batch[9/1000], Loss: 170903888.0000
Current Batch :9
Batch[10/1000], Loss: 118976072.0000
Current Batch :10
Batch[11/1000], Loss: 149275072.0000
Current Batch :11
Batch[12/1000], Loss: 136106080.0000
Current Batch :12
Batch[13/1000], Loss: 109839824.0000
Current Batch :13
Batch[14/1000], Loss: 133335616.0000
Current Batch :14
Batch[15/1000], Loss: 123090328.0000
Current Batch :15
Batch[16/1000], Loss: 134814032.0000
Current Batch :16
Batch[17/1000], Loss: 76128176.000